Scroll a specific element?


In [5]:

var scrollClient = (selector, up = false) => {
    // scroll to bottom of messages
    return client
    .execute((selector, up = false) => {
        if(selector === '' || selector === false) {
            window.scroll(window.scrollX, window.scrollY + (up ? -100000 : 100000));
            return;
        }
        var people = document.evaluate(
            selector,
            document, null,
            XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
        people.scrollTop = people.scrollTop + (up ? -100000 : 100000);
    }, selector, up)
};
module.exports = scrollClient;
scrollClient;


[Function: scrollClient]

Get all elements using XPath?



In [3]:
var getAllXPathBrowser = (selector) => {

    var getArrayXPath = (selector, ctx) => {
        if(typeof ctx == 'undefined') {
            ctx = document;
        }
        var iterator = document.evaluate(
            selector,
            ctx, null,
            XPathResult.ORDERED_NODE_ITERATOR_TYPE, null);
        var co = []; var m;
        while ((m = iterator.iterateNext()) && co.push(m.nodeValue || m));
        return co;
    };

    var getObjectXPath = (object, ctx) => {
        var result = {};
        // TODO: parse object
        for(var p in object) {
            if(object.hasOwnProperty(p)) {
                result[p] = resolveXPathQueries(object[p], ctx);
            }
        }
        return result;
    }

    var resolveXPathQueries = (selector, ctx) => {
        var inputArr = selector;
        if(!Array.isArray(inputArr)) {
            inputArr = [inputArr];
        }

        // parse array of string and objects
        var result = [], previousResults;
        for(var i = 0; i < inputArr.length; i++) {
            if(typeof inputArr[i] == 'string') {
                if(typeof previousResults != 'undefined') {
                    result = result.concat(previousResults);
                }
                // store in case the next query is a mapped object
                previousResults = getArrayXPath(inputArr[i], ctx);
            } else if (typeof inputArr[i] == 'object') {
                if(typeof previousResults != 'undefined') {
                    previousResults = previousResults.map(p => 
                                                getObjectXPath(inputArr[i], p));
                    result = result.concat(previousResults);
                    previousResults = undefined;
                } else {
                    result[result.length] = getObjectXPath(inputArr[i], ctx);
                }
            } else if (Array.isArray(inputArr[i])) {
                result = result.concat(resolveXPathQueries(inputArr[i], ctx));
            } else if (typeof inputArr[i] == 'function') {
                // TODO: what to do here?
            }
        }
        if(typeof previousResults != 'undefined') {
            result = result.concat(previousResults);
        }    

        if(!Array.isArray(selector) && result.length == 1) {
            return result[0];
        }
        return result;
    };
    return resolveXPathQueries(selector);
}
// TODO: better way to do this for selenium versus browser?
var getAllXPath = (selector) => client.execute(getAllXPathBrowser, selector).then(r => r.value).catch(e => console.log(e));
if (typeof client.getAllXPath == 'undefined') {
    client.addCommand('getAllXPath', getAllXPath);
}
module.exports = getAllXPath;
getAllXPath;


[Function: getAllXPath]

Get all elements until?



In [7]:
var importer = require('../Core');

var getAllXPath, scrollClient;
var getAllUntil = (scrollableSelector,
                   dataSelector,
                   set = [],
                   compare = (a, b) => m == e,
                   cb = (i) => i < 3, i = 0) => {
    return client
    .then(() => importer.interpretAll(['all xpath elements', 'scroll specific element']))
    .then(r => {
        getAllXPath = r[0].runInNewContext({client});
        scrollClient = r[1].runInNewContext({client});
        return getAllXPath(dataSelector);
    })
    .then(r => {
        var newPosts = r
            .filter(e => set
                .filter(m => compare(e, m)).length === 0);
        set = newPosts.concat(set);
        return newPosts.length > 0
            ? scrollClient(scrollableSelector)
            .pause(3000)
            .then(() => cb(i)
            ? getAllUntil(scrollableSelector, dataSelector, set, compare, cb, i + 1)
            : set)
            : Promise.resolve(set);
    })
    .catch(e => console.log(e))
};
if (typeof client.getAllUntil == 'undefined') {
    client.addCommand('getAllUntil', getAllUntil);
}
module.exports = getAllUntil;
getAllUntil;




[Function: getAllUntil]